# Computing definite integrals

In this short tutorial, we will show how it is possible to use heyoka.py to compute definite integrals.

Consider the integral

$$
\int_A^B f\left( t \right)\,dt, \tag{1}
$$

where $f\left( t \right)$ is a differentiable function of $t$ and $A, B\in\mathbb{R}$. This integral can be seen as as the solution of the time-dependent differential equation

$$
\frac{dx}{dt} = f\left( t \right), \tag{2}
$$

where $x$ is a dummy state variable. Indeed, the integration of (2) by quadrature between $t=A$ and $t=B$ yields:

$$
x\left(B\right) - x\left(A\right) = \int_A^B f\left(t\right) \, dt.
$$

Note that we are always free to choose $x\left( A \right) = 0$, because the dynamics of $x$ in (2) does not depend on the value of $x$ itself. Thus, provided that we set up a numerical integration of (2) in which

* we set $t=A$ as initial time coordinate,
* we set $x\left( A \right) = 0$ as initial condition,

then the definite integral (1) is the value of $x$ at $t=B$.

## Examples

Let us start easy:

In [1]:
import heyoka as hy, numpy as np

x = hy.make_vars("x")

# Integrate sin(t) between 0 and pi.
ta = hy.taylor_adaptive([(x, hy.sin(hy.time))], [0.0])
ta.propagate_until(np.pi)

# Print the result.
ta.state[0]

2.0

As expected, $\int_0^\pi \sin t\, dt = 2$. Let's try to change the integration range:

In [2]:
# Reset the state.
ta.state[0] = 0

# New integration limits: from 1 to 2.
ta.time = 1
ta.propagate_until(2.0)

ta.state[0]

0.9564491424152821

Indeed, $\int_1^2 \sin t\, dt = \cos 1 - \cos 2 = 0.9564491424152821\ldots$.

Let us try with a more complicated function:

$$
\int_\sqrt{2}^\sqrt{3} \frac{\sin \left( \cos t \right)\cdot \operatorname{erf}{t}}{\log\left(\sqrt{t}\right)}\,dt.
$$

In [3]:
ta = hy.taylor_adaptive(
    [(x, hy.sin(hy.cos(hy.time)) * hy.erf(hy.time) / hy.log(hy.sqrt(hy.time)))],
    [0.0],
    time=np.sqrt(2),
)
ta.propagate_until(np.sqrt(3))
ta.state[0]

0.012382281847117892

The result matches the value produced by [Wolfram Alpha](https://www.wolframalpha.com/input/?i=N%5BIntegrate%5BSin%5BCos%5Bx%5D%5D*Erf%5Bx%5D%2FLog%5BSqrt%5Bx%5D%5D%2C+%7Bx%2C+Sqrt%5B2%5D%2C+Sqrt%5B3%5D%7D%5D%2C+16%5D).

Note that, since heyoka.py supports integration backwards in time, flipping around the integration limits also works as expected:

In [4]:
ta.state[0] = 0
ta.time = np.sqrt(3)
ta.propagate_until(np.sqrt(2))
ta.state[0]

-0.012382281847117866

Let us also perform the integration in extended precision:

In [5]:
ta = hy.taylor_adaptive(
    [(x, hy.sin(hy.cos(hy.time)) * hy.erf(hy.time) / hy.log(hy.sqrt(hy.time)))],
    [np.longdouble(0)],
    time=np.sqrt(np.longdouble(3)),
    fp_type=np.longdouble,
)
ta.propagate_until(np.sqrt(np.longdouble(2)))
ta.state[0]

-0.012382281847117883605

## Limitations and caveats

This method for the computation of definite integrals inherits all the peculiarities and caveats of heyoka.py. For instance, the computation will fail if the derivative of $f\left( t \right)$ becomes infinite within the integration interval:

In [6]:
# Cannot compute the area of a semi-circle!
ta = hy.taylor_adaptive([(x, hy.sqrt(1 - hy.time**2))], [0.0], time=-1.0)
ta.propagate_until(1.0)
ta.state[0]

nan